In [5]:
import sys
from mido import Message, MidiFile, MidiTrack

# Map intervals (using semitone offsets relative to the tonic)
# Based on a C scale where "P8" (tonic) is 0 semitones (central C = MIDI 60)
INTERVAL_SEMITONE_MAP = {
    'P8': 0,    # tonic: central C
    'm2': 1,
    'M2': 2,
    'm3': 3,
    'A2': 3,    # augmented 2nd, enharmonically same as m3
    'M3': 4,
    'P4': 5,
    'A4': 6,
    'd5': 6,
    'P5': 7,
    'm6': 8,
    'M6': 9,
    'd7': 9,
    'm7': 10,
    'A6': 10,
    'M7': 11
}

# The encoded scale degrees in the graph file (by their graph encoding order)
ENCODED_SCALE_DEGREES = [
    'A2', 'm2', 'P8', 'A6',
    'm7', 'M2', 'm6', 'M7',
    'm3', 'M3', 'P5', 'd7',
    'P4', 'M6', 'A4', 'd5'
]

def parse_graphs_file(filename):
    """
    Parse a file containing one or more graphs.
    Each graph starts with a line beginning with "N=" and is in the following format:
      N=50
      X:
      <node values (space-separated, may span multiple lines)>
      E:
      <N*N edge values (space-separated, may span multiple lines)>
    Returns a list of tuples (N, node_values, matrix) for each graph.
    """
    with open(filename, "r") as f:
        lines = [line.strip() for line in f if line.strip()]
    
    graphs_data = []
    i = 0
    while i < len(lines):
        if lines[i].startswith("N="):
            current_chunk = []
            # Collect lines until the next "N=" (if any) or end-of-file.
            while i < len(lines) and (not (lines[i].startswith("N=") and current_chunk)):
                current_chunk.append(lines[i])
                i += 1
            graphs_data.append(parse_single_graph(current_chunk))
        else:
            i += 1
    return graphs_data

def parse_single_graph(lines):
    """
    Parse a single graph from the provided lines.
    """
    try:
        N = int(lines[0].split("=")[1])
    except Exception as e:
        print("Error reading the number of nodes:", e)
        sys.exit(1)
    
    try:
        x_index = lines.index("X:") + 1
        e_index = lines.index("E:")
    except ValueError as e:
        print("Error: File format must include 'X:' and 'E:' markers.", e)
        sys.exit(1)
    
    # Parse node values
    x_values = []
    for line in lines[x_index:e_index]:
        x_values.extend(line.split())
    if len(x_values) != N:
        print(f"Error: Expected {N} node values, but found {len(x_values)}")
        sys.exit(1)
    node_values = [int(val) for val in x_values]
    
    # Parse edge matrix values
    edge_numbers = []
    for line in lines[e_index+1:]:
        edge_numbers.extend(line.split())
    if len(edge_numbers) != N * N:
        print(f"Error: Expected {N*N} edge values, but found {len(edge_numbers)}")
        sys.exit(1)
    
    matrix = []
    for i in range(N):
        row = [int(edge_numbers[i * N + j]) for j in range(N)]
        matrix.append(row)
    
    return N, node_values, matrix

def build_note_sequence(N, node_values, matrix):
    """
    Traverse the graph to generate a note sequence.
    Assumptions:
      - Start at node 0.
      - For each node, follow the unique outgoing edge to a node with a higher index (upper-triangular).
      - Stop when no such edge exists.
    Returns a list of node indices.
    """
    sequence = []
    visited = set()
    current = 0
    while current not in visited:
        visited.add(current)
        sequence.append(current)
        # Find the next node: any j > current with a nonzero edge weight.
        next_nodes = [j for j in range(current + 1, N) if matrix[current][j] != 0]
        if not next_nodes:
            break
        current = min(next_nodes)
    return sequence

def generate_midi(note_sequence, node_values, output_file, note_duration=480, velocity=64):
    """
    Generate a MIDI file from the note sequence.
    For each node in the sequence:
      - If the node's value is 51, treat it as a rest (add delay).
      - Otherwise, use the node's value as an index into ENCODED_SCALE_DEGREES to get the interval,
        then map that interval to a semitone offset (using INTERVAL_SEMITONE_MAP).
      - MIDI note = 60 (central C) + semitone offset.
    """
    mid = MidiFile()
    track = MidiTrack()
    mid.tracks.append(track)
    
    current_time = 0  # Accumulate delay for rests.
    for idx in note_sequence:
        node_val = node_values[idx]
        if node_val == 51:
            current_time += note_duration
        else:
            try:
                interval = ENCODED_SCALE_DEGREES[node_val]
            except IndexError:
                print(f"Warning: node value {node_val} is out of range.")
                continue
            if interval not in INTERVAL_SEMITONE_MAP:
                print(f"Warning: interval '{interval}' not found in mapping.")
                continue
            offset = INTERVAL_SEMITONE_MAP[interval]
            midi_note = 60 + offset
            # Add a note-on event with the accumulated delay.
            track.append(Message('note_on', note=midi_note, velocity=velocity, time=current_time))
            # Add a note-off event after note_duration ticks.
            track.append(Message('note_off', note=midi_note, velocity=velocity, time=note_duration))
            current_time = 0
    mid.save(output_file)
    print(f"MIDI file saved as {output_file}")

In [ ]:
if len(sys.argv) < 2:
        print("Usage: python generate_midi.py <graph_file.txt>")
        sys.exit(1)
    
filename = 

graphs = parse_graphs_file
N, node_values, matrix = parse_graph_file(filename)
sequence = build_note_sequence(N, node_values, matrix)
generate_midi(sequence, node_values)

Error: expected 196 edge values, but found 1108


SystemExit: 1

/root/miniforge3/envs/digress/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3558: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [6]:

    
filename = 'generated_samples1.txt'
graphs = parse_graphs_file(filename)
if not graphs:
    print("No graphs found in file.")
    sys.exit(1)

# Process each graph in the file and generate a MIDI file for each.
for idx, (N, node_values, matrix) in enumerate(graphs):
    print(f"Processing graph {idx} with {N} nodes...")
    note_seq = build_note_sequence(N, node_values, matrix)
    output_midi = f"graph_output_{idx}.mid"
    generate_midi(note_seq, node_values, output_midi)

Processing graph 0 with 14 nodes...
MIDI file saved as graph_output_0.mid
Processing graph 1 with 17 nodes...
MIDI file saved as graph_output_1.mid
Processing graph 2 with 24 nodes...
MIDI file saved as graph_output_2.mid
